In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm

Reading the CSVs into DATAFRAMES

In [5]:
#read yelp_tip.csv dataset
df_tip = pd.read_csv("yelp_tip.csv")

In [6]:
#read yelp_business.csv dataset
df_business = pd.read_csv('yelp_business.csv')

In [7]:
#read yelp_user.csv dataset
df_users = pd.read_csv('yelp_user.csv')

In [8]:
#read yelp_review.csv dataset
df_review = pd.read_csv('yelp_review.csv')

BUSINESSES

In [9]:
#filter business df by keeping all that are over 4 stars with more than 50 reviews
df_businessNeed = df_business[(df_business['stars'] >= 4.0) & (df_business['review_count'] > 50)]

#delete the index column
df_businessNeed.reset_index(inplace=True)
del df_businessNeed['index']

# To limit our search to just Restaurants:
# Loop through each row and get the categories column value (a string)
# For each category, check if the word 'Restaurant' or 'food' is in there and if not, drop that entire line in top_bs
count = 0;
for ind, row in df_businessNeed.iterrows():
    if (('Restaurants' in row['categories']) or ('Food' in row['categories'])):
        count = count + 1
    else:
        count = count
        df_businessNeed.drop(ind, inplace = True)
        
        
# Drop unnecessary columns neighborhood and postal code. And categories since we've used it
df_businessNeed.drop(['neighborhood','postal_code', 'categories'], axis = 1, inplace = True)

#save the business we need to a new csv file for future use
df_businessNeed.to_csv('businessFinal.csv', sep='\t')

/Users/Ethan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Ethan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
#This is the final filtered version of business df
df_businessFinal = pd.read_csv("businessFinal.csv", sep='\t')
df_businessFinal.head()

,Unnamed: 0,business_id,name,address,city,state,latitude,longitude,stars,review_count,is_open
0,1,FXHfcFVEfI1vVngW2gVOpw,"""Bampot House of Tea & Board Games""","""201 Harbord Street""",Toronto,ON,43.661582,-79.408878,4.0,55,1
1,4,5q6Xh-UcJa78bp6dzyaE7w,"""Duck Donuts""","""1710 Kenilworth Ave, Ste 220""",Charlotte,NC,35.202624,-80.844419,4.5,373,1
2,5,W1Yr6c2XDx_RBjb6WsV-aQ,"""Divine Cafe at the Springs Preserve""","""333 S Valley View Blvd""",Las Vegas,NV,36.170706,-115.190288,4.0,140,1
3,6,PJ-VbAtIOso1dqd2frQqqg,"""Donut Tyme""","""4268 E Charleston Blvd""",Las Vegas,NV,36.159328,-115.082179,4.5,76,1
4,7,mLP-jIIRdoaYM7fJqqFurw,"""Napoli Pizzeria""","""2006 Murray Ave""",Pittsburgh,PA,40.434399,-79.922983,4.0,53,1


REVIEWS and TIPS

In [11]:
#drop columns that are not useful for df_review
df_review.drop(['useful', 'funny', 'cool', 'date'], axis = 1, inplace = True)

#drop date column for df_tip
df_tip = df_tip.drop(['date'], axis = 1)

USERS

In [12]:
#dropping all the columns we don't need.
df_users = df_users.drop(['name', 'yelping_since','friends','useful','funny','cool','fans','compliment_hot','compliment_more',
                          'compliment_profile','compliment_cute','compliment_list','compliment_note','compliment_plain','compliment_cool'
                         ,'compliment_funny','compliment_writer','compliment_photos'],axis = 1)

#keep all the users whose write more than 100 reviews.
df_users = df_users[df_users['review_count'] >= 100]

#drop all none elite users
df_users = df_users[df_users.elite.str.contains("None") == False]

Merge BUSINESS and REVIEWS based on business_id

In [13]:
#merge business and reviews on their business_id
df_review_business = pd.merge(df_review, df_businessFinal, on = "business_id", how='outer')

#drop all rows with NaN
df_review_business = df_review_business.dropna(axis=0, how='any')
df_review_business.head()

,review_id,user_id,business_id,stars_x,text,Unnamed: 0,name,address,city,state,latitude,longitude,stars_y,review_count,is_open
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,Super simple place but amazing nonetheless. It...,9211.0,"""Wilensky's""","""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,4.0,84.0,1.0
1,vm1b1keOzwHjtGZEPPuYXA,xYciRtVZ1PW4IxSX4oJ1aw,AEx2SYEUJmTxVVB18LlCwA,5,*The shop was featured on Mind of a Chef with ...,9211.0,"""Wilensky's""","""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,4.0,84.0,1.0
2,SXwA9KZ-Nc_hMARk_3cJ7g,5Ymfsf9fAYz-Ds_p0xawVQ,AEx2SYEUJmTxVVB18LlCwA,5,"As its name suggests, Wilensky's is a lunch co...",9211.0,"""Wilensky's""","""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,4.0,84.0,1.0
3,jUzausdZ_ujqe_n8BlBj-g,DVOOF0Z627DyrZ4XKQbTgA,AEx2SYEUJmTxVVB18LlCwA,5,"Stopped by for the Wilensky's Special, loved i...",9211.0,"""Wilensky's""","""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,4.0,84.0,1.0
4,oCRDwF3tszAkeszSfxwthg,5JoKz3mU42Cp906KRXDwJw,AEx2SYEUJmTxVVB18LlCwA,4,"I have to admit, I am a fan. Just entering the...",9211.0,"""Wilensky's""","""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,4.0,84.0,1.0


Merge BUSINESS and TIPS based on business_id

In [14]:
#merge business and tips on their business_id
df_tip_business = pd.merge(df_tip, df_businessFinal, on = "business_id", how='outer')

#drop all rows with NaN
df_tip_business = df_tip_business.dropna(axis=0, how='any')
df_tip_business.head()

,text,likes,business_id,user_id,Unnamed: 0,name,address,city,state,latitude,longitude,stars,review_count,is_open
77,Homemade pasta is the best in the area,0.0,k7WRPbDd7rztjHcGGkEjlw,ulQ8Nyj7jCUR8M83SUMoRQ,9828.0,"""Tillie's Restaurant & Chantillie Lounge""","""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0,58.0,1.0
78,"Excellent service, staff is dressed profession...",0.0,k7WRPbDd7rztjHcGGkEjlw,ulQ8Nyj7jCUR8M83SUMoRQ,9828.0,"""Tillie's Restaurant & Chantillie Lounge""","""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0,58.0,1.0
79,Ordered a fish sandwich carry out. It was deli...,0.0,k7WRPbDd7rztjHcGGkEjlw,RaqaRvK2VEnh2SWipHZs9Q,9828.0,"""Tillie's Restaurant & Chantillie Lounge""","""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0,58.0,1.0
80,Could go for some Lasagna! Excellent!,0.0,k7WRPbDd7rztjHcGGkEjlw,WBgCnzfOCA02JcYTp6APyg,9828.0,"""Tillie's Restaurant & Chantillie Lounge""","""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0,58.0,1.0
81,Eat a delicious dinner and order a pizza to ta...,0.0,k7WRPbDd7rztjHcGGkEjlw,igVuIqd3nstyjWunAr3d2A,9828.0,"""Tillie's Restaurant & Chantillie Lounge""","""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0,58.0,1.0


Merge REVIEWS and TIPS with their business matched

In [15]:
# df_RT_business = pd.merge(df_tip_business, df_review_business, on = "business_id", how='outer')
# df_RT_business

Merge USERS and REVIEWS based on user_id

In [16]:
#merge business and users on their user_id
df_review_users = pd.merge(df_review_business, df_users, on = "user_id", how='outer')

#drop all rows with NaN
df_review_users = df_review_users.dropna(axis=0, how='any')
df_review_users.head()

,review_id,user_id,business_id,stars_x,text,Unnamed: 0,name,address,city,state,latitude,longitude,stars_y,review_count_x,is_open,review_count_y,elite,average_stars
3,vm1b1keOzwHjtGZEPPuYXA,xYciRtVZ1PW4IxSX4oJ1aw,AEx2SYEUJmTxVVB18LlCwA,5.0,*The shop was featured on Mind of a Chef with ...,9211.0,"""Wilensky's""","""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,4.0,84.0,1.0,177.0,"2015, 2017, 2016",3.41
4,VCBuSkhbv6z_sQCRBV5lBw,xYciRtVZ1PW4IxSX4oJ1aw,s2I_Ni76bjJNK9yG60iD-Q,3.0,"3.5\/5\n\n*Don't go for brunch, go for the pas...",7203.0,"""Maison Christian Faure""","""355 Place Royale""",Montréal,QC,45.503544,-73.554469,4.5,397.0,1.0,177.0,"2015, 2017, 2016",3.41
5,ufI413zxZqGwIBIInnM5Rg,xYciRtVZ1PW4IxSX4oJ1aw,0W4lkclzZThpx3V65bVgig,4.0,*Schwartz's has become a Montreal's famous lan...,5406.0,"""Schwartz's""","""3895 Boulevard Saint-Laurent""",Montréal,QC,45.516373,-73.577537,4.0,1953.0,1.0,177.0,"2015, 2017, 2016",3.41
6,MxyGz9JB3_VUAjSW10pEdQ,xYciRtVZ1PW4IxSX4oJ1aw,TR0-w6VoZDAdvFQiq7P2Ug,5.0,Millie Creperie is a hidden gem tucked away in...,1233.0,"""Millie Creperie""","""161 Baldwin Street""",Toronto,ON,43.654949,-79.399671,4.0,322.0,1.0,177.0,"2015, 2017, 2016",3.41
7,Cu7qRFkAF5qXPcLyXrekYQ,xYciRtVZ1PW4IxSX4oJ1aw,e41TP5cXZqSrz50xCBJqZw,5.0,It was Insomnia that introduced me to the word...,4188.0,"""Insomnia Restaurant & Lounge""","""563 Bloor Street W""",Toronto,ON,43.665131,-79.410852,4.0,690.0,1.0,177.0,"2015, 2017, 2016",3.41


In [17]:
#drop any unnecessary columns in the df review users
df_review_users = df_review_users.drop(['Unnamed: 0', 'review_id', 'name', 'review_count_x', 'review_count_y', 'is_open', 'average_stars', 'elite'], axis=1)
df_review_users.head()

,user_id,business_id,stars_x,text,address,city,state,latitude,longitude,stars_y
3,xYciRtVZ1PW4IxSX4oJ1aw,AEx2SYEUJmTxVVB18LlCwA,5.0,*The shop was featured on Mind of a Chef with ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,4.0
4,xYciRtVZ1PW4IxSX4oJ1aw,s2I_Ni76bjJNK9yG60iD-Q,3.0,"3.5\/5\n\n*Don't go for brunch, go for the pas...","""355 Place Royale""",Montréal,QC,45.503544,-73.554469,4.5
5,xYciRtVZ1PW4IxSX4oJ1aw,0W4lkclzZThpx3V65bVgig,4.0,*Schwartz's has become a Montreal's famous lan...,"""3895 Boulevard Saint-Laurent""",Montréal,QC,45.516373,-73.577537,4.0
6,xYciRtVZ1PW4IxSX4oJ1aw,TR0-w6VoZDAdvFQiq7P2Ug,5.0,Millie Creperie is a hidden gem tucked away in...,"""161 Baldwin Street""",Toronto,ON,43.654949,-79.399671,4.0
7,xYciRtVZ1PW4IxSX4oJ1aw,e41TP5cXZqSrz50xCBJqZw,5.0,It was Insomnia that introduced me to the word...,"""563 Bloor Street W""",Toronto,ON,43.665131,-79.410852,4.0


In [18]:
#reset the index
df_review_users.reset_index()

,index,user_id,business_id,stars_x,text,address,city,state,latitude,longitude,stars_y
0,3,xYciRtVZ1PW4IxSX4oJ1aw,AEx2SYEUJmTxVVB18LlCwA,5.0,*The shop was featured on Mind of a Chef with ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,4.0
1,4,xYciRtVZ1PW4IxSX4oJ1aw,s2I_Ni76bjJNK9yG60iD-Q,3.0,"3.5\/5\n\n*Don't go for brunch, go for the pas...","""355 Place Royale""",Montréal,QC,45.503544,-73.554469,4.5
2,5,xYciRtVZ1PW4IxSX4oJ1aw,0W4lkclzZThpx3V65bVgig,4.0,*Schwartz's has become a Montreal's famous lan...,"""3895 Boulevard Saint-Laurent""",Montréal,QC,45.516373,-73.577537,4.0
3,6,xYciRtVZ1PW4IxSX4oJ1aw,TR0-w6VoZDAdvFQiq7P2Ug,5.0,Millie Creperie is a hidden gem tucked away in...,"""161 Baldwin Street""",Toronto,ON,43.654949,-79.399671,4.0
4,7,xYciRtVZ1PW4IxSX4oJ1aw,e41TP5cXZqSrz50xCBJqZw,5.0,It was Insomnia that introduced me to the word...,"""563 Bloor Street W""",Toronto,ON,43.665131,-79.410852,4.0
5,8,xYciRtVZ1PW4IxSX4oJ1aw,crstB-H5rOfbXhV8pX0e6g,4.0,*Opened by the same owners as Kingyo Izakaya. ...,"""421 College Street""",Toronto,ON,43.656458,-79.406981,4.5
6,9,xYciRtVZ1PW4IxSX4oJ1aw,ejKXczjEkm1gzyRdblQm8w,3.0,This is the first time for me to eat a Vegetar...,"""147 Spadina Avenue""",Toronto,ON,43.648068,-79.395629,4.0
7,10,xYciRtVZ1PW4IxSX4oJ1aw,cQK9M2JAwETQnnBoYyua5A,1.0,*the tables and seats were dirty and greasy. Y...,"""249 Victoria Street""",Toronto,ON,43.655737,-79.379002,4.0
8,11,xYciRtVZ1PW4IxSX4oJ1aw,6EVBc9kdc3Hd8KZkLVPnGA,4.0,"You know, I can't believe I'm actually reviewi...","""384 Yonge Street, Unit 51""",Toronto,ON,43.659238,-79.382903,4.5
9,12,xYciRtVZ1PW4IxSX4oJ1aw,Bf2fuqWbHd3L-X69FSMvmg,3.0,*restaurant was tiny and at best it can sit ar...,"""277 Augusta Ave""",Toronto,ON,43.655915,-79.402315,4.0


Merge USERS and tips based on user_id

In [19]:
#merge business and tips on their business_id
df_tip_user = pd.merge(df_tip, df_businessFinal, on = "business_id", how='outer')

#drop all rows with NaN
df_tip_user = df_tip_business.dropna(axis=0, how='any')
df_tip_user.head()

,text,likes,business_id,user_id,Unnamed: 0,name,address,city,state,latitude,longitude,stars,review_count,is_open
77,Homemade pasta is the best in the area,0.0,k7WRPbDd7rztjHcGGkEjlw,ulQ8Nyj7jCUR8M83SUMoRQ,9828.0,"""Tillie's Restaurant & Chantillie Lounge""","""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0,58.0,1.0
78,"Excellent service, staff is dressed profession...",0.0,k7WRPbDd7rztjHcGGkEjlw,ulQ8Nyj7jCUR8M83SUMoRQ,9828.0,"""Tillie's Restaurant & Chantillie Lounge""","""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0,58.0,1.0
79,Ordered a fish sandwich carry out. It was deli...,0.0,k7WRPbDd7rztjHcGGkEjlw,RaqaRvK2VEnh2SWipHZs9Q,9828.0,"""Tillie's Restaurant & Chantillie Lounge""","""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0,58.0,1.0
80,Could go for some Lasagna! Excellent!,0.0,k7WRPbDd7rztjHcGGkEjlw,WBgCnzfOCA02JcYTp6APyg,9828.0,"""Tillie's Restaurant & Chantillie Lounge""","""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0,58.0,1.0
81,Eat a delicious dinner and order a pizza to ta...,0.0,k7WRPbDd7rztjHcGGkEjlw,igVuIqd3nstyjWunAr3d2A,9828.0,"""Tillie's Restaurant & Chantillie Lounge""","""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0,58.0,1.0


In [20]:
#drop any unnecessary columns in df tip user
df_tip_user = df_tip_user.drop(['Unnamed: 0', 'likes', 'name', 'review_count', 'is_open'], axis=1)
df_tip_user.head()

,text,business_id,user_id,address,city,state,latitude,longitude,stars
77,Homemade pasta is the best in the area,k7WRPbDd7rztjHcGGkEjlw,ulQ8Nyj7jCUR8M83SUMoRQ,"""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0
78,"Excellent service, staff is dressed profession...",k7WRPbDd7rztjHcGGkEjlw,ulQ8Nyj7jCUR8M83SUMoRQ,"""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0
79,Ordered a fish sandwich carry out. It was deli...,k7WRPbDd7rztjHcGGkEjlw,RaqaRvK2VEnh2SWipHZs9Q,"""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0
80,Could go for some Lasagna! Excellent!,k7WRPbDd7rztjHcGGkEjlw,WBgCnzfOCA02JcYTp6APyg,"""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0
81,Eat a delicious dinner and order a pizza to ta...,k7WRPbDd7rztjHcGGkEjlw,igVuIqd3nstyjWunAr3d2A,"""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0


In [21]:
#reset the index 
df_tip_user.reset_index()

,index,text,business_id,user_id,address,city,state,latitude,longitude,stars
0,77,Homemade pasta is the best in the area,k7WRPbDd7rztjHcGGkEjlw,ulQ8Nyj7jCUR8M83SUMoRQ,"""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0
1,78,"Excellent service, staff is dressed profession...",k7WRPbDd7rztjHcGGkEjlw,ulQ8Nyj7jCUR8M83SUMoRQ,"""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0
2,79,Ordered a fish sandwich carry out. It was deli...,k7WRPbDd7rztjHcGGkEjlw,RaqaRvK2VEnh2SWipHZs9Q,"""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0
3,80,Could go for some Lasagna! Excellent!,k7WRPbDd7rztjHcGGkEjlw,WBgCnzfOCA02JcYTp6APyg,"""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0
4,81,Eat a delicious dinner and order a pizza to ta...,k7WRPbDd7rztjHcGGkEjlw,igVuIqd3nstyjWunAr3d2A,"""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0
5,82,Very traditional Italian food. Delicious!,k7WRPbDd7rztjHcGGkEjlw,0GHmCNC5DG3Z16R2H-bVLQ,"""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0
6,83,Loved it!,k7WRPbDd7rztjHcGGkEjlw,suRd6rt0E8TMz5Dv5G4SxQ,"""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0
7,84,Love this place try the cheese ravioli good se...,k7WRPbDd7rztjHcGGkEjlw,w4dcWDJ6KXv7fp8s-jhnXg,"""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0
8,85,The Veal Parmesan and homemade pasta is fantas...,k7WRPbDd7rztjHcGGkEjlw,PQmKbmxFNnmODEPPkXml-g,"""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0
9,86,"Best Italian food in McKeesport,cheap ,great s...",k7WRPbDd7rztjHcGGkEjlw,itm_AZHE3RtU5GgyayGpyg,"""308 36th St""",McKeesport,PA,40.327975,-79.839877,4.0


Merge REVIEWS and TIPS with their user matched

In [22]:
#merge review user and tip user
df_Final = pd.merge(df_review_users, df_tip_user, on = "business_id", how='outer')

In [23]:
#drop any row with NaN value
df_Final = df_Final.dropna(axis=0, how='any')
df_Final.head()

,user_id_x,business_id,stars_x,text_x,address_x,city_x,state_x,latitude_x,longitude_x,stars_y,text_y,user_id_y,address_y,city_y,state_y,latitude_y,longitude_y,stars
0,xYciRtVZ1PW4IxSX4oJ1aw,AEx2SYEUJmTxVVB18LlCwA,5.0,*The shop was featured on Mind of a Chef with ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,4.0,All you need to know is salami and soda.,TOJJE4oWFTet2t8xU4LVCQ,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,4.0
1,xYciRtVZ1PW4IxSX4oJ1aw,AEx2SYEUJmTxVVB18LlCwA,5.0,*The shop was featured on Mind of a Chef with ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,4.0,"FYI - there is no tipping, if you do, it's don...",6yNwk5i7LtEmW3dULoaKog,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,4.0
2,xYciRtVZ1PW4IxSX4oJ1aw,AEx2SYEUJmTxVVB18LlCwA,5.0,*The shop was featured on Mind of a Chef with ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,4.0,C'est tout de même incroyable que j'apprenne v...,imK7v4l8CH_9r7ReMipA8g,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,4.0
3,xYciRtVZ1PW4IxSX4oJ1aw,AEx2SYEUJmTxVVB18LlCwA,5.0,*The shop was featured on Mind of a Chef with ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,4.0,Basically a grab and go! Simple sandwiches and...,vGohRKrqPgCJak3Jp6nGMA,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,4.0
4,xYciRtVZ1PW4IxSX4oJ1aw,AEx2SYEUJmTxVVB18LlCwA,5.0,*The shop was featured on Mind of a Chef with ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,4.0,Ask for Ms. Sharon!!!,GHoG4X4FY8D8L563zzPX5w,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,4.0


In [24]:
#drop any unnecessary columns in final df
df_Final = df_Final.drop(['user_id_x', 'user_id_y', 'city_y', 'state_y', 'latitude_y', 'longitude_y', 'stars', 'stars_y', 'address_y'], axis=1)
df_Final.head()

,business_id,stars_x,text_x,address_x,city_x,state_x,latitude_x,longitude_x,text_y
0,AEx2SYEUJmTxVVB18LlCwA,5.0,*The shop was featured on Mind of a Chef with ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,All you need to know is salami and soda.
1,AEx2SYEUJmTxVVB18LlCwA,5.0,*The shop was featured on Mind of a Chef with ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,"FYI - there is no tipping, if you do, it's don..."
2,AEx2SYEUJmTxVVB18LlCwA,5.0,*The shop was featured on Mind of a Chef with ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,C'est tout de même incroyable que j'apprenne v...
3,AEx2SYEUJmTxVVB18LlCwA,5.0,*The shop was featured on Mind of a Chef with ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,Basically a grab and go! Simple sandwiches and...
4,AEx2SYEUJmTxVVB18LlCwA,5.0,*The shop was featured on Mind of a Chef with ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,Ask for Ms. Sharon!!!


In [25]:
#we decide to drop tips for now for a clearer dataframe
df_Final = df_Final.drop(['text_y'], axis=1)

df_Final = df_Final.drop_duplicates()
df_Final.head()

,business_id,stars_x,text_x,address_x,city_x,state_x,latitude_x,longitude_x
0,AEx2SYEUJmTxVVB18LlCwA,5.0,*The shop was featured on Mind of a Chef with ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859
16,AEx2SYEUJmTxVVB18LlCwA,5.0,Get the special with swiss along with a chocol...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859
32,AEx2SYEUJmTxVVB18LlCwA,5.0,This is a great little spot for a quick light ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859
48,AEx2SYEUJmTxVVB18LlCwA,4.0,Le 4 étoiles ici est pour l'expérience globale...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859
64,AEx2SYEUJmTxVVB18LlCwA,3.0,Family owned restaurant that's been around sin...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859


In [26]:
#only keep the restaurants that are 5-star
df_Final = df_Final[df_Final['stars_x'] == 5.0]
df_Final.head()

,business_id,stars_x,text_x,address_x,city_x,state_x,latitude_x,longitude_x
0,AEx2SYEUJmTxVVB18LlCwA,5.0,*The shop was featured on Mind of a Chef with ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859
16,AEx2SYEUJmTxVVB18LlCwA,5.0,Get the special with swiss along with a chocol...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859
32,AEx2SYEUJmTxVVB18LlCwA,5.0,This is a great little spot for a quick light ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859
112,AEx2SYEUJmTxVVB18LlCwA,5.0,I love this Montreal staple... It appears to b...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859
160,AEx2SYEUJmTxVVB18LlCwA,5.0,I couldn't believe how much I loved this sandw...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859


In [27]:
# read the business hour csv file
df_hours = pd.read_csv('yelp_business_hours.csv')

In [28]:
# reset Final's index
df_Final.reset_index()

# merge Final and hours based on their business_id
df_Final = pd.merge(df_Final, df_hours, on = 'business_id', how = 'outer')
df_Final.head()

,business_id,stars_x,text_x,address_x,city_x,state_x,latitude_x,longitude_x,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,AEx2SYEUJmTxVVB18LlCwA,5.0,*The shop was featured on Mind of a Chef with ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None
1,AEx2SYEUJmTxVVB18LlCwA,5.0,Get the special with swiss along with a chocol...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None
2,AEx2SYEUJmTxVVB18LlCwA,5.0,This is a great little spot for a quick light ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None
3,AEx2SYEUJmTxVVB18LlCwA,5.0,I love this Montreal staple... It appears to b...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None
4,AEx2SYEUJmTxVVB18LlCwA,5.0,I couldn't believe how much I loved this sandw...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None


In [29]:
#drop any row with NaN values
df_Final = df_Final.dropna(axis=0, how='any')
df_Final.head()

,business_id,stars_x,text_x,address_x,city_x,state_x,latitude_x,longitude_x,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,AEx2SYEUJmTxVVB18LlCwA,5.0,*The shop was featured on Mind of a Chef with ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None
1,AEx2SYEUJmTxVVB18LlCwA,5.0,Get the special with swiss along with a chocol...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None
2,AEx2SYEUJmTxVVB18LlCwA,5.0,This is a great little spot for a quick light ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None
3,AEx2SYEUJmTxVVB18LlCwA,5.0,I love this Montreal staple... It appears to b...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None
4,AEx2SYEUJmTxVVB18LlCwA,5.0,I couldn't believe how much I loved this sandw...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None


In [30]:
# reset the index to business_id
df_Final.set_index('business_id')

,stars_x,text_x,address_x,city_x,state_x,latitude_x,longitude_x,monday,tuesday,wednesday,thursday,friday,saturday,sunday
business_id,,,,,,,,,,,,,,
AEx2SYEUJmTxVVB18LlCwA,5.0,*The shop was featured on Mind of a Chef with ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None
AEx2SYEUJmTxVVB18LlCwA,5.0,Get the special with swiss along with a chocol...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None
AEx2SYEUJmTxVVB18LlCwA,5.0,This is a great little spot for a quick light ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None
AEx2SYEUJmTxVVB18LlCwA,5.0,I love this Montreal staple... It appears to b...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None
AEx2SYEUJmTxVVB18LlCwA,5.0,I couldn't believe how much I loved this sandw...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None
AEx2SYEUJmTxVVB18LlCwA,5.0,I can't believe this place has only 65 reviews...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None
AEx2SYEUJmTxVVB18LlCwA,5.0,Their special sandwich is simplest sandwich ev...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None
AEx2SYEUJmTxVVB18LlCwA,5.0,"Founded in the Mile End in 1932, and in the cu...","""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None
AEx2SYEUJmTxVVB18LlCwA,5.0,Yum. Read everyone's reviews and follow their ...,"""34 Avenue Fairmount Ouest""",Montréal,QC,45.523333,-73.594859,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,10:0-16:0,None


In [31]:
# extract the Final dataframe to a csv file
df_Final.to_csv('df_Final.csv', sep='\t')

In [32]:
df_Final = pd.read_csv('df_Final.csv', sep='\t')

In [33]:
df_Final.reset_index()

var = df_Final['business_id'].nunique()
print(var)

7670


In [34]:
df_Final = df_Final.groupby('business_id').agg({'stars_x':'first', 'text_x': ', '.join, 'address_x':'first', 'city_x':'first', 'state_x':'first', 'latitude_x':'first', 'longitude_x':'first', 'monday':'first', 'tuesday':'first', 'wednesday':'first', 'thursday':'first', 'friday':'first', 'saturday':'first', 'sunday':'first',}).reset_index()
df_Final.head()

,business_id,stars_x,text_x,address_x,city_x,state_x,latitude_x,longitude_x,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,--9e1ONYQuAa-CB_Rrw7Tw,5.0,There's a reason why Emeril Lagasse is such a ...,"""3355 Las Vegas Blvd S""",Las Vegas,NV,36.123183,-115.169190,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0
1,--cZ6Hhc9F7VkKXxHMVZSQ,5.0,"Listen, I could write all kinds of words and t...","""1408 E Blvd""",Charlotte,NC,35.199853,-80.844820,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,12:0-21:0
2,-050d_XIor1NpCuWkbIVaQ,5.0,Love this spot - it's pretty close to the conv...,"""801 N 1st St""",Phoenix,AZ,33.456696,-112.072327,None,14:30-6:30,14:30-6:30,14:30-6:30,14:30-6:30,14:30-6:30,None
3,-1VaIJza42Hjev6ukacCNg,5.0,"The Surprise location is our normal stop, and ...","""12245 W Santa Fe Dr""",El Mirage,AZ,33.617436,-112.324297,11:0-20:0,11:0-20:0,11:0-20:0,11:0-20:0,11:0-21:0,11:0-21:0,9:0-16:0
4,-1xuC540Nycht_iWFeJ-dw,5.0,"Excellent poulet, mais les frites étaient un p...","""115 Rue Rachel E""",Montréal,QC,45.519310,-73.580749,6:30-21:0,6:30-21:0,6:30-21:0,6:30-22:0,6:30-22:0,6:30-22:0,6:30-21:0


In [35]:
df_Final.set_index('business_id')

,stars_x,text_x,address_x,city_x,state_x,latitude_x,longitude_x,monday,tuesday,wednesday,thursday,friday,saturday,sunday
business_id,,,,,,,,,,,,,,
--9e1ONYQuAa-CB_Rrw7Tw,5.0,There's a reason why Emeril Lagasse is such a ...,"""3355 Las Vegas Blvd S""",Las Vegas,NV,36.123183,-115.169190,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0
--cZ6Hhc9F7VkKXxHMVZSQ,5.0,"Listen, I could write all kinds of words and t...","""1408 E Blvd""",Charlotte,NC,35.199853,-80.844820,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,12:0-21:0
-050d_XIor1NpCuWkbIVaQ,5.0,Love this spot - it's pretty close to the conv...,"""801 N 1st St""",Phoenix,AZ,33.456696,-112.072327,None,14:30-6:30,14:30-6:30,14:30-6:30,14:30-6:30,14:30-6:30,None
-1VaIJza42Hjev6ukacCNg,5.0,"The Surprise location is our normal stop, and ...","""12245 W Santa Fe Dr""",El Mirage,AZ,33.617436,-112.324297,11:0-20:0,11:0-20:0,11:0-20:0,11:0-20:0,11:0-21:0,11:0-21:0,9:0-16:0
-1xuC540Nycht_iWFeJ-dw,5.0,"Excellent poulet, mais les frites étaient un p...","""115 Rue Rachel E""",Montréal,QC,45.519310,-73.580749,6:30-21:0,6:30-21:0,6:30-21:0,6:30-22:0,6:30-22:0,6:30-22:0,6:30-21:0
-3oxnPPPU3YoxO9M1I2idg,5.0,My boyfriend and I had heard about Eklectic Pi...,"""1859 S Stapley Dr, Ste 105-3""",Mesa,AZ,33.379912,-111.806297,11:0-21:0,11:0-21:0,11:0-21:0,11:0-21:0,11:0-22:0,11:0-22:0,11:0-21:0
-3zffZUHoY8bQjGfPSoBKQ,5.0,Our first fancy shmancy dinner in Vegas was at...,"""3600 S Las Vegas Blvd""",Las Vegas,NV,36.111990,-115.177001,17:30-22:0,17:30-22:0,17:30-22:0,17:30-22:0,17:30-22:0,17:30-22:0,17:30-22:0
-4TMQnQJW1yd6NqGRDvAeA,5.0,Chocolate wrapped in velvet rolled up in a Sin...,"""6936 E Main St""",Scottsdale,AZ,33.493364,-111.931241,16:0-22:0,17:0-22:0,17:0-22:0,17:0-22:0,17:0-22:0,17:0-22:0,11:0-15:0
-5NXoZeGBdx3Bdk70tuyCw,5.0,REALLY love this place!!!!!!!!!!!!!!!!!!!!!!! ...,"""202 E University Ave, Ste C""",Urbana,IL,40.117695,-88.205874,11:0-20:0,11:0-20:0,11:0-20:0,11:0-20:0,11:0-21:0,11:0-20:0,11:0-20:0


In [36]:
df_Final = df_Final.drop(['stars_x', 'address_x'], axis = 1)

In [37]:
df_Final.columns = ['business_id', 'reviews', 'city', 'state', 'latitude', 'longitude', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']



In [38]:
df_Final.head()

,business_id,reviews,city,state,latitude,longitude,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,--9e1ONYQuAa-CB_Rrw7Tw,There's a reason why Emeril Lagasse is such a ...,Las Vegas,NV,36.123183,-115.169190,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0
1,--cZ6Hhc9F7VkKXxHMVZSQ,"Listen, I could write all kinds of words and t...",Charlotte,NC,35.199853,-80.844820,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,12:0-21:0
2,-050d_XIor1NpCuWkbIVaQ,Love this spot - it's pretty close to the conv...,Phoenix,AZ,33.456696,-112.072327,None,14:30-6:30,14:30-6:30,14:30-6:30,14:30-6:30,14:30-6:30,None
3,-1VaIJza42Hjev6ukacCNg,"The Surprise location is our normal stop, and ...",El Mirage,AZ,33.617436,-112.324297,11:0-20:0,11:0-20:0,11:0-20:0,11:0-20:0,11:0-21:0,11:0-21:0,9:0-16:0
4,-1xuC540Nycht_iWFeJ-dw,"Excellent poulet, mais les frites étaient un p...",Montréal,QC,45.519310,-73.580749,6:30-21:0,6:30-21:0,6:30-21:0,6:30-22:0,6:30-22:0,6:30-22:0,6:30-21:0


In [39]:
df_Final.to_csv('df_Final.csv', sep='\t')

In [40]:
df_Final_BusinessID = pd.read_csv('df_Final.csv', sep='\t')
df_Final_BusinessID.head()

,Unnamed: 0,business_id,reviews,city,state,latitude,longitude,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,0,--9e1ONYQuAa-CB_Rrw7Tw,There's a reason why Emeril Lagasse is such a ...,Las Vegas,NV,36.123183,-115.169190,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0
1,1,--cZ6Hhc9F7VkKXxHMVZSQ,"Listen, I could write all kinds of words and t...",Charlotte,NC,35.199853,-80.844820,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,12:0-21:0
2,2,-050d_XIor1NpCuWkbIVaQ,Love this spot - it's pretty close to the conv...,Phoenix,AZ,33.456696,-112.072327,None,14:30-6:30,14:30-6:30,14:30-6:30,14:30-6:30,14:30-6:30,None
3,3,-1VaIJza42Hjev6ukacCNg,"The Surprise location is our normal stop, and ...",El Mirage,AZ,33.617436,-112.324297,11:0-20:0,11:0-20:0,11:0-20:0,11:0-20:0,11:0-21:0,11:0-21:0,9:0-16:0
4,4,-1xuC540Nycht_iWFeJ-dw,"Excellent poulet, mais les frites étaient un p...",Montréal,QC,45.519310,-73.580749,6:30-21:0,6:30-21:0,6:30-21:0,6:30-22:0,6:30-22:0,6:30-22:0,6:30-21:0


In [41]:
df_Final_BusinessID = df_Final_BusinessID.drop(['reviews', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday'], axis=1)
df_Final_BusinessID.head()

,Unnamed: 0,business_id,city,state,latitude,longitude
0,0,--9e1ONYQuAa-CB_Rrw7Tw,Las Vegas,NV,36.123183,-115.169190
1,1,--cZ6Hhc9F7VkKXxHMVZSQ,Charlotte,NC,35.199853,-80.844820
2,2,-050d_XIor1NpCuWkbIVaQ,Phoenix,AZ,33.456696,-112.072327
3,3,-1VaIJza42Hjev6ukacCNg,El Mirage,AZ,33.617436,-112.324297
4,4,-1xuC540Nycht_iWFeJ-dw,Montréal,QC,45.519310,-73.580749


In [42]:
df_Final_BusinessID.to_csv('df_Final_BusinessID.csv', sep='\t')

In [43]:
df_Final_BusinessID2 = pd.read_csv('df_Final_BusinessID.csv', sep='\t')
df_Final_BusinessID2.head()

,Unnamed: 0,Unnamed: 0.1,business_id,city,state,latitude,longitude
0,0,0,--9e1ONYQuAa-CB_Rrw7Tw,Las Vegas,NV,36.123183,-115.169190
1,1,1,--cZ6Hhc9F7VkKXxHMVZSQ,Charlotte,NC,35.199853,-80.844820
2,2,2,-050d_XIor1NpCuWkbIVaQ,Phoenix,AZ,33.456696,-112.072327
3,3,3,-1VaIJza42Hjev6ukacCNg,El Mirage,AZ,33.617436,-112.324297
4,4,4,-1xuC540Nycht_iWFeJ-dw,Montréal,QC,45.519310,-73.580749


In [44]:
df_Business2 = pd.read_csv('yelp_business.csv')

df_Business2 = df_Business2.drop(['neighborhood', 'address', 'postal_code', 'stars', 'is_open', 'categories'], axis=1)
df_Business2.head()

,business_id,name,city,state,latitude,longitude,review_count
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",Ahwatukee,AZ,33.330690,-111.978599,22
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",McMurray,PA,40.291685,-80.104900,11
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",Phoenix,AZ,33.524903,-112.115310,18
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",Tempe,AZ,33.383147,-111.964725,9
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",Cuyahoga Falls,OH,41.119535,-81.475690,116


In [45]:
df_Final_BusinessID2 = pd.merge(df_Final_BusinessID2, df_Business2, on = "business_id", how='outer')
df_Final_BusinessID2.head()

,Unnamed: 0,Unnamed: 0.1,business_id,city_x,state_x,latitude_x,longitude_x,name,city_y,state_y,latitude_y,longitude_y,review_count
0,0.0,0.0,--9e1ONYQuAa-CB_Rrw7Tw,Las Vegas,NV,36.123183,-115.169190,"""Delmonico Steakhouse""",Las Vegas,NV,36.123183,-115.169190,1451
1,1.0,1.0,--cZ6Hhc9F7VkKXxHMVZSQ,Charlotte,NC,35.199853,-80.844820,"""Pio Pio""",Charlotte,NC,35.199853,-80.844820,317
2,2.0,2.0,-050d_XIor1NpCuWkbIVaQ,Phoenix,AZ,33.456696,-112.072327,"""Matt's Big Breakfast""",Phoenix,AZ,33.456696,-112.072327,700
3,3.0,3.0,-1VaIJza42Hjev6ukacCNg,El Mirage,AZ,33.617436,-112.324297,"""Rio Mirage Cafe Y Cantina""",El Mirage,AZ,33.617436,-112.324297,176
4,4.0,4.0,-1xuC540Nycht_iWFeJ-dw,Montréal,QC,45.519310,-73.580749,"""Romados""",Montréal,QC,45.519310,-73.580749,433


In [46]:
df_Final_BusinessID2 = df_Final_BusinessID2.dropna(axis=0, how='any')
df_Final_BusinessID2.head()

,Unnamed: 0,Unnamed: 0.1,business_id,city_x,state_x,latitude_x,longitude_x,name,city_y,state_y,latitude_y,longitude_y,review_count
0,0.0,0.0,--9e1ONYQuAa-CB_Rrw7Tw,Las Vegas,NV,36.123183,-115.169190,"""Delmonico Steakhouse""",Las Vegas,NV,36.123183,-115.169190,1451
1,1.0,1.0,--cZ6Hhc9F7VkKXxHMVZSQ,Charlotte,NC,35.199853,-80.844820,"""Pio Pio""",Charlotte,NC,35.199853,-80.844820,317
2,2.0,2.0,-050d_XIor1NpCuWkbIVaQ,Phoenix,AZ,33.456696,-112.072327,"""Matt's Big Breakfast""",Phoenix,AZ,33.456696,-112.072327,700
3,3.0,3.0,-1VaIJza42Hjev6ukacCNg,El Mirage,AZ,33.617436,-112.324297,"""Rio Mirage Cafe Y Cantina""",El Mirage,AZ,33.617436,-112.324297,176
4,4.0,4.0,-1xuC540Nycht_iWFeJ-dw,Montréal,QC,45.519310,-73.580749,"""Romados""",Montréal,QC,45.519310,-73.580749,433


In [47]:
df_Final_BusinessID2 = df_Final_BusinessID2.drop(['city_y', 'state_y', 'latitude_y', 'longitude_y'], axis=1)

In [48]:
df_Final_BusinessID2.columns = ['unnamed1', 'unnamed2', 'business_id', 'city', 'state', 'latitude', 'longitude', 'name', 'review_count']

In [49]:
df_Final_BusinessID2 = df_Final_BusinessID2.drop(['unnamed1', 'unnamed2'], axis=1)

In [50]:
df_Final_BusinessID2.head()

,business_id,city,state,latitude,longitude,name,review_count
0,--9e1ONYQuAa-CB_Rrw7Tw,Las Vegas,NV,36.123183,-115.169190,"""Delmonico Steakhouse""",1451
1,--cZ6Hhc9F7VkKXxHMVZSQ,Charlotte,NC,35.199853,-80.844820,"""Pio Pio""",317
2,-050d_XIor1NpCuWkbIVaQ,Phoenix,AZ,33.456696,-112.072327,"""Matt's Big Breakfast""",700
3,-1VaIJza42Hjev6ukacCNg,El Mirage,AZ,33.617436,-112.324297,"""Rio Mirage Cafe Y Cantina""",176
4,-1xuC540Nycht_iWFeJ-dw,Montréal,QC,45.519310,-73.580749,"""Romados""",433


In [54]:
df_Final_BusinessID2.reset_index(drop=True, inplace=True)

In [57]:
df_Final_BusinessID2.head()

,business_id,city,state,latitude,longitude,name,review_count
0,--9e1ONYQuAa-CB_Rrw7Tw,Las Vegas,NV,36.123183,-115.169190,"""Delmonico Steakhouse""",1451
1,--cZ6Hhc9F7VkKXxHMVZSQ,Charlotte,NC,35.199853,-80.844820,"""Pio Pio""",317
2,-050d_XIor1NpCuWkbIVaQ,Phoenix,AZ,33.456696,-112.072327,"""Matt's Big Breakfast""",700
3,-1VaIJza42Hjev6ukacCNg,El Mirage,AZ,33.617436,-112.324297,"""Rio Mirage Cafe Y Cantina""",176
4,-1xuC540Nycht_iWFeJ-dw,Montréal,QC,45.519310,-73.580749,"""Romados""",433


In [60]:
df_Final_BusinessID2.to_csv('df_Final_BusinessID2.csv')

In [61]:
var2 = df_Final_BusinessID2['city'].nunique()
print(var2)

230


In [62]:
df_Final_BusinessID2['city'].value_counts()

Las Vegas               1539
Phoenix                  854
Toronto                  780
Scottsdale               443
Charlotte                443
Pittsburgh               393
Montréal                 310
Cleveland                234
Henderson                214
Tempe                    212
Mesa                     211
Chandler                 203
Madison                  200
Gilbert                  143
Glendale                 116
Edinburgh                 84
Mississauga               72
Peoria                    50
Lakewood                  49
Champaign                 47
Markham                   43
North Las Vegas           37
Goodyear                  34
Surprise                  33
Huntersville              30
Concord                   27
Scarborough               27
Avondale                  24
Cleveland Heights         24
Cave Creek                23
                        ... 
Westmount                  1
Mount Horeb                1
Harrisburg                 1
Laveen Village

In [64]:
vc = df_Final_BusinessID2['city'].value_counts()
u  = [i not in set(vc[vc==1].index) for i in df_Final_BusinessID2['city']]
df_Final_BusinessID2 = df_Final_BusinessID2[u]
df_Final_BusinessID2

,business_id,city,state,latitude,longitude,name,review_count
0,--9e1ONYQuAa-CB_Rrw7Tw,Las Vegas,NV,36.123183,-115.169190,"""Delmonico Steakhouse""",1451
1,--cZ6Hhc9F7VkKXxHMVZSQ,Charlotte,NC,35.199853,-80.844820,"""Pio Pio""",317
2,-050d_XIor1NpCuWkbIVaQ,Phoenix,AZ,33.456696,-112.072327,"""Matt's Big Breakfast""",700
3,-1VaIJza42Hjev6ukacCNg,El Mirage,AZ,33.617436,-112.324297,"""Rio Mirage Cafe Y Cantina""",176
4,-1xuC540Nycht_iWFeJ-dw,Montréal,QC,45.519310,-73.580749,"""Romados""",433
5,-3oxnPPPU3YoxO9M1I2idg,Mesa,AZ,33.379912,-111.806297,"""Eklectic Pie - Mesa""",127
6,-3zffZUHoY8bQjGfPSoBKQ,Las Vegas,NV,36.111990,-115.177001,"""Michael Mina""",590
7,-4TMQnQJW1yd6NqGRDvAeA,Scottsdale,AZ,33.493364,-111.931241,"""The House Brasserie""",353
8,-5NXoZeGBdx3Bdk70tuyCw,Urbana,IL,40.117695,-88.205874,"""Po' Boys Restaurant""",56
9,-5XuRAfrjEiMN77J4gMQZQ,Charlotte,NC,35.202356,-80.844867,"""Berrybrook Farm Natural Foods""",51


In [65]:
df_Final_BusinessID2['city'].value_counts()

Las Vegas                 1539
Phoenix                    854
Toronto                    780
Scottsdale                 443
Charlotte                  443
Pittsburgh                 393
Montréal                   310
Cleveland                  234
Henderson                  214
Tempe                      212
Mesa                       211
Chandler                   203
Madison                    200
Gilbert                    143
Glendale                   116
Edinburgh                   84
Mississauga                 72
Peoria                      50
Lakewood                    49
Champaign                   47
Markham                     43
North Las Vegas             37
Goodyear                    34
Surprise                    33
Huntersville                30
Scarborough                 27
Concord                     27
Cleveland Heights           24
Avondale                    24
Cave Creek                  23
                          ... 
Woodbridge                   3
Spring V

In [66]:
vc = df_Final_BusinessID2['city'].value_counts()
u  = [i not in set(vc[vc==2].index) for i in df_Final_BusinessID2['city']]
df_Final_BusinessID2 = df_Final_BusinessID2[u]
df_Final_BusinessID2

,business_id,city,state,latitude,longitude,name,review_count
0,--9e1ONYQuAa-CB_Rrw7Tw,Las Vegas,NV,36.123183,-115.169190,"""Delmonico Steakhouse""",1451
1,--cZ6Hhc9F7VkKXxHMVZSQ,Charlotte,NC,35.199853,-80.844820,"""Pio Pio""",317
2,-050d_XIor1NpCuWkbIVaQ,Phoenix,AZ,33.456696,-112.072327,"""Matt's Big Breakfast""",700
3,-1VaIJza42Hjev6ukacCNg,El Mirage,AZ,33.617436,-112.324297,"""Rio Mirage Cafe Y Cantina""",176
4,-1xuC540Nycht_iWFeJ-dw,Montréal,QC,45.519310,-73.580749,"""Romados""",433
5,-3oxnPPPU3YoxO9M1I2idg,Mesa,AZ,33.379912,-111.806297,"""Eklectic Pie - Mesa""",127
6,-3zffZUHoY8bQjGfPSoBKQ,Las Vegas,NV,36.111990,-115.177001,"""Michael Mina""",590
7,-4TMQnQJW1yd6NqGRDvAeA,Scottsdale,AZ,33.493364,-111.931241,"""The House Brasserie""",353
8,-5NXoZeGBdx3Bdk70tuyCw,Urbana,IL,40.117695,-88.205874,"""Po' Boys Restaurant""",56
9,-5XuRAfrjEiMN77J4gMQZQ,Charlotte,NC,35.202356,-80.844867,"""Berrybrook Farm Natural Foods""",51


In [67]:
df_Final_BusinessID2['city'].value_counts()

Las Vegas             1539
Phoenix                854
Toronto                780
Scottsdale             443
Charlotte              443
Pittsburgh             393
Montréal               310
Cleveland              234
Henderson              214
Tempe                  212
Mesa                   211
Chandler               203
Madison                200
Gilbert                143
Glendale               116
Edinburgh               84
Mississauga             72
Peoria                  50
Lakewood                49
Champaign               47
Markham                 43
North Las Vegas         37
Goodyear                34
Surprise                33
Huntersville            30
Concord                 27
Scarborough             27
Avondale                24
Cleveland Heights       24
Cave Creek              23
                      ... 
Broadview Heights        4
Streetsboro              4
Bridgeville              4
Independence             4
Tega Cay                 4
Sun City                 4
B

In [95]:
df = df_Final_BusinessID2['city'].value_counts()
df

Las Vegas             1539
Phoenix                854
Toronto                780
Scottsdale             443
Charlotte              443
Pittsburgh             393
Montréal               310
Cleveland              234
Henderson              214
Tempe                  212
Mesa                   211
Chandler               203
Madison                200
Gilbert                143
Glendale               116
Edinburgh               84
Mississauga             72
Peoria                  50
Lakewood                49
Champaign               47
Markham                 43
North Las Vegas         37
Goodyear                34
Surprise                33
Huntersville            30
Concord                 27
Scarborough             27
Avondale                24
Cleveland Heights       24
Cave Creek              23
                      ... 
Broadview Heights        4
Streetsboro              4
Bridgeville              4
Independence             4
Tega Cay                 4
Sun City                 4
B

In [96]:
df.to_csv('cities.csv', sep='\t')

In [73]:
df.column

dtype('int64')

In [87]:
df_City = pd.DataFrame(df_Final_BusinessID2.city.value_counts().reset_index())
df_City.columns = ['city', 'count']
df_City.head()

,city,count
0,Las Vegas,1539
1,Phoenix,854
2,Toronto,780
3,Scottsdale,443
4,Charlotte,443


In [84]:
df_City.to_csv('cities.csv', sep='\t')

In [103]:
df_Pop = pd.read_csv('mergedPop.csv', encoding="ISO-8859-1")
df_Pop

,Name,2018 Population
0,Toronto,2600000
1,Montreal,1600000
2,Calgary,1019942
3,Ottawa,812129
4,Edmonton,712391
5,Mississauga,668549
6,North York,636000
7,Winnipeg,632063
8,Vancouver,600000
9,Scarborough,600000


In [104]:
df_Pop.columns = ['city', 'population']
df_Pop.head()

,city,population
0,Toronto,2600000
1,Montreal,1600000
2,Calgary,1019942
3,Ottawa,812129
4,Edmonton,712391


In [105]:
df_Pop['city'] = df_Pop['city'].apply(lambda x: x.rsplit(' ',1)[0])
df_Pop

,city,population
0,Toronto,2600000
1,Montreal,1600000
2,Calgary,1019942
3,Ottawa,812129
4,Edmonton,712391
5,Mississauga,668549
6,North,636000
7,Winnipeg,632063
8,Vancouver,600000
9,Scarborough,600000


In [106]:
df_FinalPop = pd.merge(df_City, df_Pop, on = 'city', how = 'outer')
df_FinalPop

,city,count,population
0,Las Vegas,1539.0,632912.0
1,Las Vegas,1539.0,13285.0
2,Phoenix,854.0,1615017.0
3,Phoenix,854.0,1947.0
4,Phoenix,854.0,2281.0
5,Phoenix,854.0,4587.0
6,Toronto,780.0,2600000.0
7,Toronto,780.0,119.0
8,Toronto,780.0,265.0
9,Toronto,780.0,4851.0


In [107]:
df_FinalPop = df_FinalPop.dropna(axis=0, how='any')
df_FinalPop

,city,count,population
0,Las Vegas,1539.0,632912.0
1,Las Vegas,1539.0,13285.0
2,Phoenix,854.0,1615017.0
3,Phoenix,854.0,1947.0
4,Phoenix,854.0,2281.0
5,Phoenix,854.0,4587.0
6,Toronto,780.0,2600000.0
7,Toronto,780.0,119.0
8,Toronto,780.0,265.0
9,Toronto,780.0,4851.0


In [113]:
df_FinalPop['population'] = [float(num) for num in df_FinalPop['population']]

/Users/Ethan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [117]:
# df_FinalPop.sort_values(by='population', ascending=False)
# df_FinalPop

df_FinalPop = df_FinalPop.groupby('population', as_index=False).max()
df_FinalPop

,population,city,count
0,27.0,Gilbert,143.0
1,34.0,Concord,27.0
2,41.0,Surprise,33.0
3,50.0,Elyria,3.0
4,52.0,Sun City,4.0
5,58.0,Aurora,6.0
6,78.0,Kent,7.0
7,81.0,Cleveland,234.0
8,86.0,Verona,4.0
9,119.0,Toronto,780.0


In [128]:
df_FinalPop = df_FinalPop.drop_duplicates(subset='city', keep="last")
df_FinalPop.head()

,population,city,count
4,52.0,Sun City,4.0
19,182.0,Thornhill,13.0
76,860.0,Woodmere,5.0
118,2005.0,Valley View,3.0
119,2021.0,North York,14.0


In [130]:
df_FinalPop.reset_index(drop = True, inplace = True)
df_FinalPop.head()

,population,city,count
0,52.0,Sun City,4.0
1,182.0,Thornhill,13.0
2,860.0,Woodmere,5.0
3,2005.0,Valley View,3.0
4,2021.0,North York,14.0


In [132]:
import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

/Users/Ethan/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [133]:
outcome, predictors = patsy.dmatrices('count ~ population', df_FinalPop)
mod = sm.OLS(outcome, predictors)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  count   R-squared:                       0.376
Model:                            OLS   Adj. R-squared:                  0.370
Method:                 Least Squares   F-statistic:                     61.49
Date:                Thu, 22 Mar 2018   Prob (F-statistic):           4.50e-12
Time:                        13:28:40   Log-Likelihood:                -673.28
No. Observations:                 104   AIC:                             1351.
Df Residuals:                     102   BIC:                             1356.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     13.6029     17.058      0.797      0.4